In [9]:
import numpy as np
import cv2
import os

### credit: https://github.com/alishobeiri/Monocular-Video-Odometery
class MonoVideoOdometery(object):
    def __init__(self, 
                img_file_path,
#                 pose_file_path,
                focal_length = 172.98992,
                pp = (163.33639726024606, 134.99537889030861),
                lk_params=dict(winSize  = (21,21),
                               criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)),
                detector=cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)):
        '''
        Arguments:
            img_file_path {str} -- File path that leads to image sequences
            pose_file_path {str} -- File path that leads to true poses from image sequence
        
        Keyword Arguments:
            focal_length {float} -- Focal length of camera used in image sequence (default: {718.8560})
            pp {tuple} -- Principal point of camera in image sequence (default: {(607.1928, 185.2157)})
            lk_params {dict} -- Parameters for Lucas Kanade optical flow (default: {dict(winSize  = (21,21), criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))})
            detector {cv2.FeatureDetector} -- Most types of OpenCV feature detectors (default: {cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)})
        
        Raises:
            ValueError -- Raised when file either file paths are not correct, or img_file_path is not configured correctly
        '''

        self.file_path = img_file_path
        self.detector = detector
        self.lk_params = lk_params
        self.focal = focal_length
        self.pp = pp
        self.R = np.zeros(shape=(3, 3))
        self.t = np.zeros(shape=(3, 3))
        self.id = 0
        self.n_features = 0
        self.vid_length = len(os.listdir(self.file_path))

        try:
            if not all([".png" in x for x in os.listdir(img_file_path)]):
                raise ValueError("img_file_path is not correct and is not exclusively png files")
        except Exception as e:
            print(e)
            raise ValueError("The designated img_file_path does not exist, please check the path and try again")

#         try:
#             with open(pose_file_path) as f:
#                 self.pose = f.readlines()
#         except Exception as e:
#             print(e)
#             raise ValueError("The pose_file_path is not valid or did not lead to a txt file")
        
        self.process_frame()


    def hasNextFrame(self):
        '''Used to determine whether there are remaining frames
           in the folder to process
        
        Returns:
            bool -- Boolean value denoting whether there are still 
            frames in the folder to process
        '''
    
        return self.id < self.vid_length 


    def detect(self, img):
        '''Used to detect features and parse into useable format
        
        Arguments:
            img {np.ndarray} -- Image for which to detect keypoints on
        
        Returns:
            np.array -- A sequence of points in (x, y) coordinate format
            denoting location of detected keypoint
        '''

        p0 = self.detector.detect(img)
        
        return np.array([x.pt for x in p0], dtype=np.float32).reshape(-1, 1, 2)


    def visual_odometery(self):
        '''
        Used to perform visual odometery. If features fall out of frame
        such that there are less than 2000 features remaining, a new feature
        detection is triggered. 
        '''

        if self.n_features < 2000:
            self.p0 = self.detect(self.old_frame)


        # Calculate optical flow between frames, st holds status
        # of points from frame to frame
        self.p1, st, err = cv2.calcOpticalFlowPyrLK(self.old_frame, 
                                self.current_frame, self.p0, None, **self.lk_params)
        

        # Save the good points from the optical flow
        self.good_old = self.p0[st == 1]
        self.good_new = self.p1[st == 1]


        # If the frame is one of first two, we need to initalize
        # our t and R vectors so behavior is different
        if self.id < 2:
            E, _ = cv2.findEssentialMat(self.good_new, self.good_old, 
                                        self.focal, self.pp, cv2.RANSAC, 0.999, 1.0, None)
            _, self.R, self.t, _ = cv2.recoverPose(E, self.good_old, self.good_new,
                                                   self.R, self.t, self.focal, self.pp, None)
        else:
            E, _ = cv2.findEssentialMat(self.good_new, self.good_old, self.focal, 
                                        self.pp, cv2.RANSAC, 0.999, 1.0, None)
            _, R, t, _ = cv2.recoverPose(E, self.good_old, self.good_new, self.R.copy(), 
                                         self.t.copy(), self.focal, self.pp, None)
            absolute_scale=1
#             absolute_scale = self.get_absolute_scale()
            if (absolute_scale > 0.1 and abs(t[2][0]) > abs(t[0][0]) and abs(t[2][0]) > abs(t[1][0])):
                self.t = self.t + absolute_scale*self.R.dot(t)
                self.R = R.dot(self.R)

        # Save the total number of good features
        self.n_features = self.good_new.shape[0]


    def get_mono_coordinates(self):
        # We multiply by the diagonal matrix to fix our vector
        # onto same coordinate axis as true values
        diag = np.array([[-1, 0, 0],
                        [0, -1, 0],
                        [0, 0, -1]])
        adj_coord = np.matmul(diag, self.t)

        return adj_coord.flatten()


    def get_true_coordinates(self):
        '''Returns true coordinates of vehicle
        
        Returns:
            np.array -- Array in format [x, y, z]
        '''
        return self.true_coord.flatten()


    def get_absolute_scale(self):
        '''Used to provide scale estimation for mutliplying
           translation vectors
        
        Returns:
            float -- Scalar value allowing for scale estimation
        '''
        pose = self.pose[self.id - 1].strip().split()
        x_prev = float(pose[3])
        y_prev = float(pose[7])
        z_prev = float(pose[11])
        pose = self.pose[self.id].strip().split()
        x = float(pose[3])
        y = float(pose[7])
        z = float(pose[11])

        true_vect = np.array([[x], [y], [z]])
        self.true_coord = true_vect
        prev_vect = np.array([[x_prev], [y_prev], [z_prev]])
        
        return np.linalg.norm(true_vect - prev_vect)

    def process_frame(self):
        '''Processes images in sequence frame by frame
        '''

        if self.id < 2:
            self.old_frame = cv2.imread(self.file_path +str().zfill(6)+'.png', 0)
            self.current_frame = cv2.imread(self.file_path + str(1).zfill(6)+'.png', 0)
            self.visual_odometery()
            self.id = 2
        else:
            self.old_frame = self.current_frame
            self.current_frame = cv2.imread(self.file_path + str(self.id).zfill(6)+'.png', 0)
            self.visual_odometery()
            self.id += 1

In [11]:
import cv2 as cv
import matplotlib.pyplot as plt

img_path = 'KITTI/dataset/sequences/00/image_0/'
pose_path = 'KITTI/dataset/poses/00.txt'

focal = 718.8560
pp = (607.1928, 185.2157)
R_total = np.zeros((3, 3))
t_total = np.empty(shape=(3, 1))

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (21,21),
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 30, 0.01))


# Create some random colors
color = np.random.randint(0,255,(5000,3))

vo = MonoVideoOdometery(img_path, focal, pp, lk_params)
# vo = MonoVideoOdometery(img_path, pose_path, focal, pp, lk_params)
traj = np.zeros(shape=(600, 800, 3))

# mask = np.zeros_like(vo.current_frame)
# flag = False
while(vo.hasNextFrame()):

    frame = vo.current_frame

    # for i, (new,old) in enumerate(zip(vo.good_new, vo.good_old)):
    #     a,b = new.ravel()    
    #     c,d = old.ravel()
        
    #     if np.linalg.norm(new - old) < 10:
    #         if flag:
    #             mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
    #             frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)


    # cv.add(frame, mask)
    cv.imshow('frame', frame)
    k = cv.waitKey(1)
    if k == 27:
        break

    if k == 121:
        flag = not flag
        toggle_out = lambda flag: "On" if flag else "Off"
        print("Flow lines turned ", toggle_out(flag))
        mask = np.zeros_like(vo.old_frame)
        mask = np.zeros_like(vo.current_frame)

    vo.process_frame()

    print(vo.get_mono_coordinates())

    mono_coord = vo.get_mono_coordinates()
#     true_coord = vo.get_true_coordinates()

#     print("MSE Error: ", np.linalg.norm(mono_coord - true_coord))
    print("x: {}, y: {}, z: {}".format(*[str(pt) for pt in mono_coord]))
#     print("true_x: {}, true_y: {}, true_z: {}".format(*[str(pt) for pt in true_coord]))

    draw_x, draw_y, draw_z = [int(round(x)) for x in mono_coord]
#     true_x, true_y, true_z = [int(round(x)) for x in true_coord]

#     traj = cv.circle(traj, (true_x + 400, true_z + 100), 1, list((0, 0, 255)), 4)
    traj = cv.circle(traj, (draw_x + 400, draw_z + 100), 1, list((0, 255, 0)), 4)

#     cv.putText(traj, 'Actual Position:', (140, 90), cv.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255), 1)
    cv.putText(traj, 'Red', (270, 90), cv.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255), 1)
    cv.putText(traj, 'Estimated Odometry Position:', (30, 120), cv.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255), 1)
    cv.putText(traj, 'Green', (270, 120), cv.FONT_HERSHEY_SIMPLEX, 0.5,(0, 255, 0), 1)

    cv.imshow('trajectory', traj)
cv.imwrite("images/trajectoryK.png", traj)

cv.destroyAllWindows()

[-0.10004014  0.01533944  1.99663934]
x: -0.10004013689904241, y: 0.015339440874482543, z: 1.9966393384358025
[-1.01388115e-01 -1.28492089e-03  2.99650024e+00]
x: -0.10138811487474647, y: -0.0012849208888618142, z: 2.9965002355366632
[-0.12721897 -0.01767062  3.99603226]
x: -0.12721896888242262, y: -0.017670621917911196, z: 3.9960322639294605
[-0.13694784 -0.03193172  4.99588324]
x: -0.13694783583969675, y: -0.03193172007618072, z: 4.995883237938611
[-0.14710626 -0.05068639  5.99565575]
x: -0.1471062599383874, y: -0.050686388165468554, z: 5.9956557464847755
[-0.21312096 -0.06687815  6.99334302]
x: -0.21312096146593354, y: -0.06687815287481123, z: 6.993343015090437
[-0.21733379 -0.07428489  7.99330671]
x: -0.217333794433189, y: -0.07428489091777518, z: 7.9933067105664035
[-0.26208534 -0.10471747  8.99184122]
x: -0.2620853444691331, y: -0.1047174733239631, z: 8.991841215076622
[-0.36363559 -0.09807676  9.98664946]
x: -0.36363558764280673, y: -0.09807675747117534, z: 9.98664946243102
[-0.

[-5.83606272 -0.7634462  78.73566344]
x: -5.836062719457989, y: -0.763446196058164, z: 78.73566344481156
[-5.94758639 -0.77076218 79.72939829]
x: -5.947586385305221, y: -0.7707621810092327, z: 79.72939829290748
[-6.06012287 -0.77354968 80.72304198]
x: -6.060122871353709, y: -0.7735496771368633, z: 80.72304197611247
[-6.15668436 -0.74876566 81.71806038]
x: -6.156684364648337, y: -0.748765661034599, z: 81.71806038325806
[-6.21759739 -0.82774417 82.71307395]
x: -6.217597387915534, y: -0.8277441680422538, z: 82.71307395051596
[-6.33921216 -0.84888363 83.70542619]
x: -6.339212157638479, y: -0.8488836329440151, z: 83.70542619181383
[-6.40549628 -0.84991995 84.70322644]
x: -6.405496283625492, y: -0.8499199484729069, z: 84.70322644271191
[-6.55860035 -0.88477444 85.69082166]
x: -6.558600348201844, y: -0.884774442007819, z: 85.69082165821806
[-6.66619691 -0.88640958 86.68501495]
x: -6.666196908472768, y: -0.8864095774132176, z: 86.6850149525779
[-6.77130563 -0.89956365 87.67938869]
x: -6.771305

[ 44.11954325  -0.60193883 115.10437105]
x: 44.11954325219477, y: -0.6019388315328864, z: 115.10437104706604
[ 45.11532448  -0.57227807 115.1912041 ]
x: 45.11532448292258, y: -0.5722780653863944, z: 115.1912041023031
[ 46.11289782  -0.55512417 115.25868138]
x: 46.11289781895321, y: -0.5551241690637102, z: 115.25868138056532
[ 47.10961152  -0.54662412 115.33923909]
x: 47.109611521038495, y: -0.5466241196901334, z: 115.33923909426231
[ 48.10432272  -0.48677278 115.42271039]
x: 48.104322715201, y: -0.48677278319346806, z: 115.42271039308628
[ 49.10334276  -0.48834659 115.46694234]
x: 49.10334276425556, y: -0.48834659401572, z: 115.46694233523816
[ 50.10191431  -0.47162907 115.51769063]
x: 50.10191431147164, y: -0.47162907410166527, z: 115.51769063194254
[ 51.1007761   -0.49977767 115.55619765]
x: 51.10077609681842, y: -0.49977766612247304, z: 115.5561976513167
[ 52.10016235  -0.51685474 115.58678351]
x: 52.10016234787771, y: -0.5168547442707028, z: 115.58678350745255
[ 53.0993191   -0.497

[ 74.07228936  -0.21611535 133.69334495]
x: 74.0722893623163, y: -0.21611534856758896, z: 133.69334495245553
[ 74.34005096  -0.20769134 134.65679332]
x: 74.34005096100525, y: -0.20769133903677944, z: 134.65679332280598
[ 74.63146382  -0.18101187 135.61301858]
x: 74.6314638210584, y: -0.1810118711544103, z: 135.6130185844411
[ 74.91353041  -0.1532114  136.57201054]
x: 74.91353041096632, y: -0.15321139772490586, z: 136.5720105408906
[ 7.51769935e+01 -1.26353586e-01  1.37536306e+02]
x: 75.17699353269732, y: -0.12635358610126807, z: 137.5363060565094
[ 7.54684344e+01 -9.94334593e-02  1.38492516e+02]
x: 75.46843439099648, y: -0.09943345930833124, z: 138.49251604018835
[ 7.57597828e+01 -1.03108915e-01  1.39449126e+02]
x: 75.75978283411057, y: -0.10310891532032883, z: 139.44912597935797
[ 7.60450421e+01 -9.34113851e-02  1.40407527e+02]
x: 76.04504211343678, y: -0.09341138508590845, z: 140.4075273052469
[ 7.63154871e+01 -7.19609895e-02  1.41370024e+02]
x: 76.31548710279289, y: -0.0719609894803

[ 89.92507892   0.28500559 204.78064092]
x: 89.92507892473522, y: 0.28500559352164573, z: 204.78064092327133
[ 89.99645853   0.2876752  205.77808657]
x: 89.99645852928545, y: 0.2876751951925721, z: 205.7780865735607
[ 90.17368765   0.27692237 206.76219745]
x: 90.17368765393071, y: 0.2769223733046811, z: 206.76219744851275
[ 90.24059803   0.31227891 207.7593298 ]
x: 90.24059802620013, y: 0.31227890969721755, z: 207.7593297955044
[ 90.25901866   0.30712819 208.75914685]
x: 90.25901865541327, y: 0.3071281864220417, z: 208.7591468540052
[ 90.32183962   0.29732372 209.75712351]
x: 90.32183961522021, y: 0.29732371826397774, z: 209.75712350674465
[ 90.32672769   0.29601209 210.7571107 ]
x: 90.32672769324265, y: 0.2960120929421015, z: 210.75711069982876
[ 90.41886081   0.29769445 211.75285598]
x: 90.4188608135667, y: 0.29769444628152936, z: 211.75285597741063
[ 90.46143708   0.29673884 212.75194874]
x: 90.4614370771051, y: 0.2967388439379516, z: 212.75194874017456
[ 90.50927664   0.31660975 21

...

[ 23.57540097 458.26847462 135.3692284 ]
x: 23.57540097382951, y: 458.2684746157396, z: 135.36922840458192
[ 24.34039437 458.46068935 134.75454263]
x: 24.340394366331395, y: 458.46068934628454, z: 134.75454262567544
[ 25.11319396 458.64970486 134.14869287]
x: 25.11319395860039, y: 458.6497048643815, z: 134.14869287193505
[ 25.87065406 458.84961661 133.52717095]
x: 25.870654055980815, y: 458.84961660734604, z: 133.5271709528516
[ 26.64060375 459.03616946 132.91694517]
x: 26.640603748026635, y: 459.03616946295807, z: 132.91694517186443
[ 27.41109624 459.22279204 132.30742624]
x: 27.411096244491894, y: 459.22279203589574, z: 132.3074262401152
[ 28.17966677 459.40625589 131.69453117]
x: 28.179666767738873, y: 459.40625588659776, z: 131.69453117056307
[ 28.94551802 459.5793334  131.07524446]
x: 28.945518021048006, y: 459.5793334045121, z: 131.07524445942087
[ 29.72049659 459.76219538 130.47029015]
x: 29.720496590163183, y: 459.76219537820094, z: 130.4702901504101
[ 30.4936067  459.94316497 

[ 83.21054255 473.10590734  87.35152599]
x: 83.21054255497685, y: 473.1059073434147, z: 87.35152599214832
[ 83.97842139 473.2980531   86.74042689]
x: 83.97842138662125, y: 473.29805309589483, z: 86.7404268923976
[ 84.731615   473.49706561  86.11345528]
x: 84.73161499722912, y: 473.4970656123201, z: 86.11345527960685
[ 85.48432306 473.70676434  85.48939037]
x: 85.48432306198949, y: 473.70676433750043, z: 85.48939036799725
[ 86.24615333 473.91105325  84.87467037]
x: 86.24615333454508, y: 473.9110532451597, z: 84.87467036830218
[ 86.99585731 474.121666    84.24730587]
x: 86.99585731054064, y: 474.12166600065285, z: 84.24730586850968
[ 87.7624983  474.32236031  83.63740147]
x: 87.76249830481358, y: 474.3223603072374, z: 83.63740146517696
[ 88.53178656 474.5083608   83.02617369]
x: 88.5317865609826, y: 474.5083607989257, z: 83.02617368765699
[ 89.29604831 474.69333369  82.40836384]
x: 89.2960483100731, y: 474.6933336891408, z: 82.40836383927122
[ 90.05758903 474.88036202  81.78781881]
x: 90

[145.73744772 487.56002283  42.29513795]
x: 145.737447723262, y: 487.5600228313538, z: 42.295137946083486
[146.72478544 487.69121626  42.20596102]
x: 146.724785436201, y: 487.69121626436913, z: 42.205961022264084
[147.71221608 487.83990875  42.15237673]
x: 147.7122160765503, y: 487.83990874744916, z: 42.15237673375786
[148.70602203 487.94500971  42.18848104]
x: 148.7060220264918, y: 487.94500971327074, z: 42.18848103882102
[149.6970197  488.0420383   42.28072565]
x: 149.69701969821617, y: 488.0420382968186, z: 42.28072565173409
[150.67649975 488.18079104  42.42689851]
x: 150.67649974989394, y: 488.1807910423906, z: 42.42689850817954
[151.65395346 488.31172621  42.59254943]
x: 151.65395345941556, y: 488.3117262125248, z: 42.59254942717556
[152.61995937 488.45106065  42.8103073 ]
x: 152.61995937211398, y: 488.45106065405474, z: 42.810307297378
[153.56836199 488.56790825  43.10505993]
x: 153.56836199156584, y: 488.5679082465763, z: 43.10505992508239
[154.50409794 488.69929553  43.43237569

[203.98044015 497.787863    87.45465819]
x: 203.9804401469613, y: 497.7878629975569, z: 87.45465819405406
[204.10522249 497.98622766  88.42681087]
x: 204.10522248969784, y: 497.98622765995634, z: 88.42681087118467
[204.22568706 498.19124828  89.3981269 ]
x: 204.22568705747682, y: 498.1912482752701, z: 89.3981269037951
[204.27660551 498.40271102  90.37418572]
x: 204.27660550841435, y: 498.4027110195751, z: 90.37418572329574
[204.26614732 498.57006881  91.36002648]
x: 204.26614731650096, y: 498.570068813921, z: 91.36002647863782
[204.28460128 498.75958884  92.3417299 ]
x: 204.28460128469695, y: 498.7595888413526, z: 92.3417299013985
[204.24797306 498.93388854  93.32574107]
x: 204.24797305561992, y: 498.93388853879, z: 93.32574107425569
[204.19395168 499.11394949  94.30791201]
x: 204.19395168020807, y: 499.11394949468036, z: 94.30791200803571
[204.1125467  499.2829222   95.29016526]
x: 204.1125467031536, y: 499.28292220080647, z: 95.29016526168013
[204.03144086 499.46454015  96.27018402]


[195.83154931 512.30055832 162.52253298]
x: 195.83154930541193, y: 512.3005583241934, z: 162.52253297907075
[195.73302778 512.47349186 163.5025266 ]
x: 195.7330277786278, y: 512.4734918613701, z: 163.50252660174363
[195.58952401 512.64282459 164.47758203]
x: 195.58952401286487, y: 512.642824585498, z: 164.477582034912
[195.47948366 512.84009174 165.45173643]
x: 195.47948365646508, y: 512.8400917355913, z: 165.45173643307348
[195.37685305 513.03128224 166.42790914]
x: 195.37685304884178, y: 513.0312822357041, z: 166.42790913861555
[195.25535104 513.23734524 167.39887531]
x: 195.2553510393257, y: 513.2373452423292, z: 167.39887530640104
[195.11895656 513.42010665 168.37252556]
x: 195.11895655767464, y: 513.4201066502338, z: 168.3725255583682
[195.02991155 513.60954009 169.35037317]
x: 195.02991155097195, y: 513.609540086545, z: 169.35037317430258
[194.91185228 513.7818806  170.32831016]
x: 194.91185228169596, y: 513.7818806034932, z: 170.32831016366126
[194.80500668 513.97444923 171.3037